In [ ]:
# Install
install.packages("reticulate")

# Load reticulate
library(reticulate)

# Use the pvactools conda environment
use_condaenv("pvactools", required = TRUE)


In [5]:
py_run_string("
import subprocess

##
vcf_file = 'path/to/annotated.expression.vcf.gz'
tumor_sample_name = 'TUMOR_DNA'
hla_alleles = 'HLA-A*29:02,HLA-B*45:01,HLA-B*82:02,HLA-C*06:02'
pvac_output_dir = 'path/to/output'
normal_sample_name = 'NORMAL_DNA'
phased_vcf = 'path/to/phased.vcf.gz'
peptide_fasta = 'path/to/Homo_sapiens.GRCh38.pep.all.fa.gz'


##
pvacseq_command = [
    'pvacseq', 'run',
    vcf_file,
    tumor_sample_name,
    hla_alleles,
    'all_class_i',
    pvac_output_dir,
    '--normal-sample-name', normal_sample_name,
    '--phased-proximal-variants-vcf', phased_vcf,
    '--pass-only',
    '--allele-specific-binding-thresholds',
    '--percentile-threshold', '0.01',
    '--allele-specific-anchors',
    '--run-reference-proteome-similarity',
    '--peptide-fasta', peptide_fasta,
    '--problematic-amino-acids', 'C',
    '--downstream-sequence-length', '100',
    '--n-threads', '1'
]

print('Running pvacseq...')
subprocess.run(pvacseq_command)
")


In [7]:
# Load devtools if it's not already installed
if (!requireNamespace("devtools", quietly = TRUE)) {
  install.packages("devtools") }

# Install CrossDome if it is not already installed or needs an update
if (!("crossdome" %in% rownames(installed.packages()))) {
  devtools::install_github("antuneslab/crossdome", build_vignettes = TRUE) } 

library(crossdome)
library(ggplot2)

In [9]:
# Access Python variables
pvac_output_dir <- py$pvac_output_dir
tumor_sample_name <- py$tumor_sample_name

# Construct path and load table
file_path <- file.path(pvac_output_dir, "MHC_Class_I", paste0(tumor_sample_name, ".all_epitopes.tsv"))
table <- read.delim(file_path, sep = "\t", header = TRUE)

# Subset columns 17 and 21
table <- table[, c(17, 21)]

df <- table[nchar(table$MT.Epitope.Seq) == 9, ]


In [10]:
# Load the data from CSV file
#df <- read.csv(table)
#print(colnames(df))

# Using data from CSV file to set variables
for (i in 1:nrow(df)) {
    allele_value <- df[i,1]
    query_value <- df[i, 2]

    # Create a unique identifier for files
    file_identifier <- paste(allele_value, query_value, sep="_")
    file_identifier <- gsub("[:*]", "", file_identifier) # Remove special characters that may not be valid in file names

    # Set up the database with specific allele, with off_target
    # database <- cross_background(off_targets = off_target_value, allele = allele_value)

    # Set up the database with specific allele, withOUT off_target
    database <- cross_background(allele = allele_value)
    database

    # Compose result using the query value
    result <- cross_compose(query = query_value, background = database)
    result

    # View result - use print if running in a non-interactive environment
    #print(result@result)

    # Save each result to a separate CSV file
    result_file_name <- paste0(file_identifier, ".csv")
    write.csv(result@result, file = result_file_name, row.names = FALSE)
    print(paste0("Your results were saved as: ", file_identifier, ".csv"))
    
    
    # Calculate counts
    total_peptides <- nrow(result@result)
    significant_peptides <- sum(result@result$pvalue <= 0.05)
    
    print(paste0("From: ", total_peptides, ", there are ", significant_peptides, " significant peptides (lower than 0.05)"))


    # Optionally, collect all results in a list for further processing
    #results_list[[file_identifier]] <- result@result

    tryCatch({
    # Extracting the gene donor mRNA expression based on CR candidates
    expression_matrix <- cross_expression_matrix(result, pvalue_threshold = 0.005)
    expression_matrix

    # The heatmap presenting the gene donor expression profile
    pdf(paste0(file_identifier, "_expression_heatmap.pdf"))
    print(cross_expression_plot(object = expression_matrix))
    dev.off()
    print(paste0("Your heatmap presenting the gene donor expression profile was saved as: ",
                 file_identifier, "_expression_heatmap.pdf"))

    ## Summarizing tissue specificity across candidates

    # Bar plot summarizing the tissue-specificity groups
    pdf(paste0(file_identifier, "_tissue_specificity.pdf"))
    print(cross_tissues_plot(object = expression_matrix))
    dev.off()
    print(paste0("Your Bar plot summarizing the tissue-specificity groups was saved as: ",
                 file_identifier, "_tissue_specificity.pdf"))

    ## Displaying peptides composition across best-score candidates

    # Calculate position-specific substitution across cross-reactive candidates
    cross_result <- cross_substitution_matrix(result)
    cross_result

    # Heatmap combined with seqlogo displaying amino acid substitutions
    pdf(paste0(file_identifier, "_cross_substitution.pdf"))
    print(cross_substitution_plot(object = cross_result))
    dev.off()
    print(paste0("Your Heatmap combined with seqlogo displaying amino acid substitutions was saved as: ",
                file_identifier, "cross_substitution.pdf"))
    }, error = function(e) {
    message(paste("Skipping expression and tissue plots for", file_identifier, 
                  "because:", e$message))
})}

##------ Mon Jun 30 14:21:17 2025 ------##
[1] "Your results were saved as: HLA-A2902_LTRVTPMGW.csv"
[1] "From: 12812, there are 24 significant peptides (lower than 0.05)"


Skipping expression and tissue plots for HLA-A2902_LTRVTPMGW because: The provided epitopes are not included on the human proteome.



##------ Mon Jun 30 14:21:25 2025 ------##
[1] "Your results were saved as: HLA-A2902_TRVTPMGWG.csv"
[1] "From: 12812, there are 2 significant peptides (lower than 0.05)"


Skipping expression and tissue plots for HLA-A2902_TRVTPMGWG because: The provided epitopes are not included on the human proteome.



##------ Mon Jun 30 14:21:26 2025 ------##
[1] "Your results were saved as: HLA-A2902_RVTPMGWGC.csv"
[1] "From: 12812, there are 6 significant peptides (lower than 0.05)"


...

Warning message:
“No shared levels found between `names(values)` of the manual scale and the data's colour values.”


[1] "Your Heatmap combined with seqlogo displaying amino acid substitutions was saved as: HLA-A2902_QKEKPIPQEcross_substitution.pdf"
##------ Mon Jun 30 14:24:03 2025 ------##
[1] "Your results were saved as: HLA-A2902_QKEKPIPQE.csv"
[1] "From: 12812, there are 32 significant peptides (lower than 0.05)"


Warning message in .local(object, ...):
“Found 2 peptides out of 2. Unmapped peptides: ”


[1] "Your heatmap presenting the gene donor expression profile was saved as: HLA-A2902_QKEKPIPQE_expression_heatmap.pdf"
[1] "Your Bar plot summarizing the tissue-specificity groups was saved as: HLA-A2902_QKEKPIPQE_tissue_specificity.pdf"


Warning message:
“No shared levels found between `names(values)` of the manual scale and the data's colour values.”
